In [ ]:
# implement Regression algorithm without using MLlib
from pyspark import SparkConf,SparkContext
import numpy as np
from pyspark.sql.functions import broadcast
params_num = 1000
mean = 0
std = 4
loss_list = []
for i in range(50):
   params = np.random.normal(mean, std, [para9ms_num, 10])
   a = np.array(range(0,params_num))[...,None]\
   params = np.hstack((params, a))
   rdd1 = sc.parallelize(params)
   rdd2 = rdd1.map(lambda x: [float(i) for i in x])
   params_df = rdd2.toDF(["A", "B", "C", "D", "E", "F", "G","H","I","J", "id"])
   data_df = spark.read.parquet("/user/s_feeds/Rihab/dis_materials/fast1_metadata.parquet")
   mapping = {col: col.replace('.','_').replace(' ', '_') for col in data_df.columns}
   from pyspark.sql.functions import col
   data_df = data_df.select([col('`'+c+'`').alias(mapping.get(c, c)) for c in data_df.columns])
   from pyspark.sql.functions import asc,abs, row_number, monotonically_increasing_id
   from pyspark.sql.window import Window
   data_df = data_df.select("out_electricity_total_energy_consumption", "in_sqft",
            "in_number_of_stories").withColumn("random_id",(row_number().over
              (Window.orderBy(monotonically_increasing_id())) - 1)% params_num)
   df = data_df.join(broadcast(params_df),data_df.random_id==params_df.id)
   df = df.withColumn("loss", abs(col("out_electricity_total_energy_consumption") - 
             col('A') * col("in_sqft") ** 3 - col('B') * col("in_sqft") ** 2 * 
             col("in_number_of_stories") - col('C') * col("in_sqft") * 
             col("in_number_of_stories") ** 2  - col('D')  * col("in_number_of_stories")
             ** 3 - col('E') * col("in_sqft") ** 2 -  col('F') * col("in_sqft") * 
             col("in_number_of_stories") -  col('G') * col("in_number_of_stories") **
             2 -  col('H') * col("in_sqft") -  col('I') * col("in_number_of_stories") -  col('J')))
   from pyspark.sql.functions import mean
   result = df.groupBy("id").agg(mean('loss').alias("mean")).sort(asc("mean"))
   r = result.head(100)
   print(r[0])
   loss_list.append("step" +str(i) +" loss:" + str(r[0].mean))
   l = [int(i.id) for i in r]
   params_id = np.take(params, l, axis=0)
   new_params = params_id[:, :-1]
   mean = new_params.mean(0)
   std = new_params.std(0)
   print("step=", i, "mean=", mean, "std=", std)

################################evaluate running time##############
import numpy as np
from pyspark.sql.functions import broadcast
import time
params_num = 1000
mean = 0
std = 4
loss_list = []
start_time = time.time()
round = 2
for i in range(round):
        params = np.random.normal(mean, std, [params_num, 10])
        a = np.array(range(0,params_num))[...,None]
        params = np.hstack((params, a))
        rdd1 = sc.parallelize(params)
        rdd2 = rdd1.map(lambda x: [float(i) for i in x])
        params_df = rdd2.toDF(["A", "B", "C", "D", "E", "F", "G","H","I","J", "id"])
        data_df = spark.read.parquet("hdfs://namenode:9000/dis_materials/fast2_metadata.parquet")
        mapping = {col: col.replace('.','_').replace(' ', '_') for col in data_df.columns}
        from pyspark.sql.functions import col
        data_df = data_df.select([col('`'+c+'`').alias(mapping.get(c, c)) for c in data_df.columns])
from pyspark.sql.functions import asc,abs, row_number, monotonically_increasing_id
from pyspark.sql.window import Window 
data_df = data_df.select("out_electricity_total_energy_consumption", "in_sqft", "in_number_of_stories").
withColumn("random_id",(row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)% params_num)
df = data_df.join(broadcast(params_df),data_df.random_id==params_df.id)
df = df.withColumn("loss", abs(col("out_electricity_total_energy_consumption") -
        col('A') * col("in_sqft") ** 3 - col('B') * col("in_sqft") ** 2 * 
        col("in_number_of_stories") - col('C') * col("in_sqft") * col("in_number_of_stories")
        ** 2  - col('D')  * col("in_number_of_stories") ** 3 - col('E') * 
        col("in_sqft") ** 2 -  col('F') * col("in_sqft") * col("in_number_of_stories") - 
        col('G') * col("in_number_of_stories") ** 2 -  col('H') * col("in_sqft") -  col('I')
        * col("in_number_of_stories") -  col('J')))
from pyspark.sql.functions import mean 
result = df.groupBy("id").agg(mean('loss').alias("mean")).sort(asc("mean"))
r = result.head(100) 
print(r[0])
loss_list.append("step" +str(i) +" loss:" + str(r[0].mean))
l = [int(i.id) for i in r]
params_id = np.take(params, l, axis=0)
new_params = params_id[:, :-1]   
mean = new_params.mean(0)  
std = new_params.std(0) 
print("step=", i, "mean=", mean, "std=", std)
print(str(round)+" round time cost:" + str(time.time() - start_time))
print(loss_list) 